In [1]:
%reload_ext autoreload 
%autoreload 2

In [2]:
import pandas as pd

In [3]:
import hopsworks

In [4]:
import src.config as config

In [5]:
import plotly_express as px

In [5]:
df = pd.read_parquet(r'D:\0_Respaldo\0_Proyectos_2024\ML_proyects\ipvn_predictor\data\transformed\ts_Bogotá.parquet')

### Test 01

In [8]:
dfp = df.tail(24)
dfp

,Fecha,Bogotá
221,2022-06-01,216.541408
222,2022-07-01,216.197662
223,2022-08-01,214.089739
224,2022-09-01,215.156856
225,2022-10-01,216.450298
226,2022-11-01,216.389542
227,2022-12-01,214.269154
228,2023-01-01,210.171998
229,2023-02-01,206.885759
230,2023-03-01,205.269388


In [13]:
# concatenar con prediction
last_month = dfp['Fecha'].tail(1)
last_month = pd.to_datetime(last_month)
next_month = last_month + pd.DateOffset(months=1)
next_month

244   2024-06-01
Name: Fecha, dtype: datetime64[ns]

In [25]:
next_month.iloc[0]

Timestamp('2024-06-01 00:00:00')

In [14]:
prediction = 203.026532

In [26]:
df_prediction = pd.DataFrame({
    'fecha':[next_month.iloc[0]],
    'prediccion':[prediction]
})

In [13]:
df_prediction = pd.DataFrame({
    'fecha':['2024-01-01', '2024-02-01'	,'2024-03-01' ,'2024-04-01', '2024-05-01' ],
    'prediccion':[ 204.226532, 203.026532, 202.106532 ,201.956532, 202.126532]
})

In [14]:
df_prediction.head()

,fecha,prediccion
0,2024-01-01,204.226532
1,2024-02-01,203.026532
2,2024-03-01,202.106532
3,2024-04-01,201.956532
4,2024-05-01,202.126532


In [7]:
project= hopsworks.login(
        project=config.HOPSWORKS_PROJECT_NAME,
        api_key_value=config.HOPSWORKS_API_KEY
    )

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/845972


In [8]:
feature_store =project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [15]:
feature_group = feature_store.get_or_create_feature_group(
                                                            name= config.FEATURE_GROUP_NAME_PREDICTIONS,
                                                            version= config.FEATURE_GROUP_VERSION ,
                                                            description= 'History predictions',
                                                            primary_key= ['fecha']
)

In [16]:
feature_group.insert(df_prediction, write_options={'wait_for_job':False})

Uploading Dataframe: 0.00% |          | Rows 0/5 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/845972/jobs/named/predictions_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2a7f3997f90>, None)

In [19]:
try:
    #create feature view
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME_PREDICTIONS,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
    
except:
    print('Feature view already existed. Skip creation')   
    
#get feature view
fature_view = feature_store.get_feature_view(
    name= config.FEATURE_VIEW_NAME_PREDICTIONS,
    version= config.FEATURE_VIEW_VERSION
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/845972/fs/841795/fv/predictions_bogota/version/2


In [35]:
# Graficar usando Plotly
fig = px.line(dfp, x='Fecha', y='Bogotá', title='Gráfico de Ciudad vs Fecha', markers=True)

# Agregar la predicción como una 'x'
fig.add_scatter(x=df_prediction['fecha'], y=df_prediction['prediccion'], mode='markers', marker=dict(symbol='x', size=10, color='red'), name='Predicción')

fig.show()

### Test 02

In [6]:
from src.inference import transform_ts_to_training_data,load_batch_time_series_from_store, load_model_from_registry, get_model_predictions, load_prediction_registry, download_prediction_registry_from_store


In [66]:
from src.plot import plot_x

In [8]:
time_series = load_batch_time_series_from_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/845972
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.34s) 
2024-07-09 19:53:33,923 WARNING: VersionWarning: Incremented version to `20`.



In [9]:
features = transform_ts_to_training_data(time_series)

In [10]:
model = load_model_from_registry()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/845972
Connected. Call `.close()` to terminate connection gracefully.


In [11]:
prediction = get_model_predictions(model, features)

[200.96753901]


In [12]:
prediction_history = download_prediction_registry_from_store()    

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/845972
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.84s) 
2024-07-09 19:55:47,704 WARNING: VersionWarning: Incremented version to `8`.



In [75]:
prediction_history.head(1)

,fecha,prediccion
2,2024-01-01,202.026532


In [76]:
prediction_history['fecha'] = pd.to_datetime(prediction_history['fecha'])
prediction_history['fecha'] = prediction_history['fecha'].dt.date 

In [77]:
prediction_history.head(1)

,fecha,prediccion
2,2024-01-01,202.026532


In [13]:
last_prediction = prediction_history['prediccion'].tail(1)

In [16]:
last_prediction.iloc[0]

201.026532

In [40]:
prediction[0]

200.9675390050369

In [43]:
if last_prediction.iloc[0] != prediction[0]:
    print(True)
else:
    print(False)

True


In [18]:
time_series

,bogota
fecha,
2023-06-01 00:00:00+00:00,205.409686
2023-07-01 00:00:00+00:00,205.825196
2023-08-01 00:00:00+00:00,205.019173
2023-09-01 00:00:00+00:00,204.711818
2023-10-01 00:00:00+00:00,204.335083
2023-11-01 00:00:00+00:00,205.451510
2023-12-01 00:00:00+00:00,205.408361
2024-01-01 00:00:00+00:00,204.164086
2024-02-01 00:00:00+00:00,203.058967


In [23]:
last_month = time_series.index[-1]

In [24]:
last_month

'2024-05-01 00:00:00+00:00'

In [25]:
last_month = pd.to_datetime(last_month)

In [26]:
next_month = last_month + pd.DateOffset(months=1)

In [58]:
df_prediction = pd.DataFrame({
    'fecha':next_month,
    'prediccion':prediction
    })   

In [73]:
df_prediction['fecha'] = pd.to_datetime(df_prediction['fecha'])
df_prediction['fecha'] = df_prediction['fecha'].dt.date 

In [74]:
df_prediction.head(1)

,fecha,prediccion
0,2024-06-01,200.967539


In [50]:
prediction_history =prediction_history.sort_values(by='fecha', ascending=True)
prediction_history

,fecha,prediccion
2,2024-01-01,202.026532
3,2024-02-01,201.026532
0,2024-03-01,202.026532
1,2024-04-01,203.026532


In [59]:
df_prediction['fecha'] = df_prediction['fecha'].dt.date

In [61]:
prediction_update = pd.concat([prediction_history, df_prediction], axis=0)


In [62]:
prediction_update

,fecha,prediccion
2,2024-01-01,202.026532
3,2024-02-01,201.026532
0,2024-03-01,202.026532
1,2024-04-01,203.026532
0,2024-06-01,200.967539


In [67]:
plot_x(time_series, prediction_update)